# Experiment CIFAR-10
> Can we get better by training on our assumptions?

In [ ]:
# default_exp experiment_cifar10_cinic10

In [ ]:
# hide
import blackhc.project.script

Import modules and functions were are going to use.

In [ ]:
# exports

import dataclasses
import traceback

from blackhc.project import is_run_from_ipython
from blackhc.project.experiment import embedded_experiments

from batchbald_redux import acquisition_functions, baseline_acquisition_functions
from batchbald_redux.experiment_data import (
    OoDDatasetConfig,
    StandardExperimentDataConfig,
)
from batchbald_redux.resnet_models import Cifar10ModelWorkshopPaperTrainer
from batchbald_redux.unified_experiment import UnifiedExperiment

In [ ]:
# exports

shared_configs = dict(
    num_validation_samples=1,
    num_pool_samples=1,
    num_training_samples=1,
    resnet18_dropout_head=False,
    ensemble_size=3,
    max_training_epochs=105,
    model_trainer_factory=Cifar10ModelWorkshopPaperTrainer,
    max_training_set=16000,
)

configs = sum(
    [
        [
            UnifiedExperiment(
                experiment_data_config=StandardExperimentDataConfig(
                    id_dataset_name=id_dataset_name,
                    id_repetitions=1,
                    initial_training_set_size=1000,
                    validation_set_size=1024,
                    validation_split_random_state=0,
                    evaluation_set_size=0,
                    add_dataset_noise=False,
                    ood_dataset_config=None,
                ),
                seed=seed + 1456,
                acquisition_function=acquisition_function,
                acquisition_size=acquisition_size,
                **shared_configs
            )
            for acquisition_function in [
                acquisition_functions.BALD,
            ]
        ]
        + [
            UnifiedExperiment(
                experiment_data_config=StandardExperimentDataConfig(
                    id_dataset_name=id_dataset_name,
                    id_repetitions=1,
                    initial_training_set_size=1000,
                    validation_set_size=1024,
                    validation_split_random_state=0,
                    evaluation_set_size=0,
                    add_dataset_noise=False,
                    ood_dataset_config=None,
                ),
                seed=seed + 8945,
                acquisition_function=acquisition_function,
                acquisition_size=acquisition_size,
                coldness=coldness,
                stochastic_mode=stochastic_mode,
                **shared_configs
            )
            for stochastic_mode in [
                acquisition_functions.StochasticMode.Power,
                acquisition_functions.StochasticMode.Softmax,
                acquisition_functions.StochasticMode.Softrank,
            ]
            for coldness in ([1/8, 1] if stochastic_mode != acquisition_functions.StochasticMode.Softrank else [1])
            for acquisition_function in [
                acquisition_functions.StochasticBALD,
            ]
        ]
        for seed in range(3)
        for id_dataset_name in ["CINIC-10"]
        for acquisition_size in [300]
    ],
    [],
)
# + [
#     UnifiedExperiment(
#         experiment_data_config=StandardExperimentDataConfig(
#             id_dataset_name=id_dataset_name,
#             id_repetitions=1,
#             initial_training_set_size=1000,
#             validation_set_size=1024,
#             validation_split_random_state=0,
#             evaluation_set_size=0,
#             add_dataset_noise=False,
#             ood_dataset_config=None,
#         ),
#         seed=seed + 1456,
#         acquisition_function=acquisition_function,
#         acquisition_size=acquisition_size,
#         num_validation_samples=1,
#         num_pool_samples=1,
#         num_training_samples=1,
#         resnet18_dropout_head=False,
#         ensemble_size=1,
#         max_training_epochs=105,
#         model_trainer_factory=Cifar10ModelWorkshopPaperTrainer,
#         max_training_set=16000,
#     )
#     for acquisition_function in [
#         baseline_acquisition_functions.BADGE,
#     ]
#     for seed in range(3)
#     for id_dataset_name in ["CIFAR-10", "CINIC-10"]
#     for acquisition_size in [300, 900]
# ]
#     UnifiedExperiment(
#         seed=seed + 8945,
#         acquisition_function=acquisition_function,
#         acquisition_size=acquisition_size,
#         num_pool_samples=num_pool_samples,
#         initial_training_set_size=1000,
#         evaluation_set_size=0,
#         max_training_set=15000,
#         temperature=temperature,
#         id_dataset_name="CIFAR-10",
#         ood_dataset_name=None,
#         ood_exposure=False,
#         id_repetitions=id_repetitions,
#         add_dataset_noise=True

#     )
#     for seed in range(5)
#     for acquisition_function in [
#         acquisition_functions.SieveBALD,
#     ]
#     for acquisition_size in [3000]
#     for num_pool_samples in [100]
#     for temperature in [1/64]
#     for id_repetitions in [1,5,10,20]
# ]
# +
#     UnifiedExperiment(
#         seed=seed + 8945,
#         acquisition_function=acquisition_function,
#         acquisition_size=acquisition_size,
#         num_pool_samples=num_pool_samples,
#         initial_training_set_size=1000,
#         evaluation_set_size=0,
#         max_training_set=15000,
#         temperature=temperature,
#         id_dataset_name="CIFAR-10",
#         ood_dataset_name=None,
#         ood_exposure=False,
#         id_repetitions=id_repetitions,
#         add_dataset_noise=True

#     )
#     for seed in range(5)
#     for acquisition_function in [
#         acquisition_functions.SoftmaxBALD,
#     ]
#     for acquisition_size in [3000]
#     for num_pool_samples in [100]
#     for temperature in [1/64]
#     for id_repetitions in [1,5,10,20]
# ] + [
#     UnifiedExperiment(
#         seed=seed + 8945,
#         acquisition_function=acquisition_function,
#         acquisition_size=acquisition_size,
#         num_pool_samples=num_pool_samples,
#         initial_training_set_size=1000,
#         evaluation_set_size=0,
#         max_training_set=15000,
#         temperature=temperature,
#         id_repetitions=id_repetitions,
#         add_dataset_noise=True,
#         id_dataset_name="CIFAR-10",
#         ood_dataset_name=None,
#         ood_exposure=False,
#     )
#     for seed in range(5)
#     for acquisition_function in [
#         acquisition_functions.BALD,
#     ]
#     for acquisition_size in [3000]
#     for num_pool_samples in [100]
#     for temperature in [0]
#     for id_repetitions in [1,5,10,20]
# ]

if not is_run_from_ipython() and __name__ == "__main__":
    for job_id, store in embedded_experiments(__file__, len(configs)):
        config = configs[job_id]
        config.seed += job_id
        print(config)
        store["config"] = dataclasses.asdict(config)
        store["log"] = {}

        try:
            config.run(store=store)
        except Exception:
            store["exception"] = traceback.format_exc()
            raise

In [ ]:
len(configs)

18

In [ ]:
import prettyprinter

prettyprinter.install_extras({"dataclasses"})
prettyprinter.pprint(configs)

[
    batchbald_redux.unified_experiment.UnifiedExperiment(
        seed=1456,
        experiment_data_config=batchbald_redux.experiment_data.StandardExperimentDataConfig(
            id_dataset_name='CIFAR-10',
            id_repetitions=1,
            initial_training_set_size=1000,
            validation_set_size=1024,
            validation_split_random_state=0,
            evaluation_set_size=0,
            add_dataset_noise=False,
            ood_dataset_config=None
        ),
        acquisition_size=300,
        max_training_set=16000,
        max_training_epochs=105,
        num_pool_samples=1,
        num_validation_samples=1,
        # class
        acquisition_function=batchbald_redux.baseline_acquisition_functions.BADGE,
        # class
        model_trainer_factory=batchbald_redux.resnet_models.Cifar10ModelWorkshopPaperTrainer,
        ensemble_size=3,
        resnet18_dropout_head=False
    ),
    batchbald_redux.unified_experiment.UnifiedExperiment(
        seed=1456,
 

In [ ]:
store = {}

configs[3].max_training_epochs = 1
configs[3].num_pool_samples = 5
configs[3].run(store)

StandardExperimentDataConfig(id_dataset_name='CIFAR-10', id_repetitions=1, initial_training_set_size=1000, validation_set_size=1024, validation_split_random_state=0, evaluation_set_size=0, add_dataset_noise=False, ood_dataset_config=None)
Files already downloaded and verified


/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

Files already downloaded and verified
Creating: StochasticBALD(
	acquisition_size=300,
	num_pool_samples=5,
	coldness=8,
	stochastic_mode=StochasticMode.Power
)
Creating: Cifar10ModelWorkshopPaperTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=1,
	max_training_epochs=1,
	resnet18_dropout_head=False
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=5
)
Training set size 1000:
NeurIPS Workshop Style
Limit schedule/max epochs updated: [40, 60], 60


  2%|1         | 1/60 [00:00<?, ?it/s]

/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/kornia/utils/helpers.py:96: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:766.)
  out1, out2 = torch.solve(input.to(dtype), A.to(dtype))


...

Epoch metrics: {'accuracy': 0.40625, 'crossentropy': 1.7282845973968506}
Epoch 18: 0.40625 worse than 0.4150390625, patience: 2/20!


[1/10]  10%|#          [00:00<?]

Epoch metrics: {'accuracy': 0.431640625, 'crossentropy': 1.7795724868774414}


Engine run is terminating due to exception: 


KeyboardInterrupt: 